## ====================== ||| =====================

## Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Cài đặt packages

In [2]:
# Install the most re version of TensorFlow to use the improved
# masking support for `tf.keras.layers.MultiHeadAttention`.
!apt install --allow-change-held-packages libcudnn8=8.6.0.163-1+cuda11.8
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
!pip install protobuf~=3.20.3
!pip install -q tensorflow_datasets
!pip install -q -U tensorflow-text tensorflow

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be DOWNGRADED:
  libcudnn8
0 upgraded, 0 newly installed, 1 downgraded, 1 to remove and 22 not upgraded.
Need to get 446 MB of archives.
After this operation, 1,441 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  libcudnn8 8.6.0.163-1+cuda11.8 [446 MB]
Fetched 446 MB in 7s (67.6 MB/s)
(Reading database ... 122518 files and directories currently installed.)
Removing libcudnn8-dev (8.7.0.84-1+cuda11.8) ...
update-alternatives: removing manually selected alternative - switching libcudnn to auto mode
dpkg: warning: downgrading libcudnn8 from 8.7.0.84-1+cuda11.8 to 8.6.0.163-1+cuda11.8
(Reading database ... 122485 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.6.0

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.7 MB/s eta 0:00:00


In [4]:
import logging
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow as tf

import tensorflow_text

## Data handling

### Prepare data

In [5]:
train_vi_sentt = []
train_en_sentt = []

In [6]:
train_vi_sen = []
train_en_sen = []

In [7]:
test_vi_sen = []
test_en_sen = []

In [8]:
# Đọc dữ liệu tiếng Việt
with open('/content/drive/MyDrive/NLP/DataTrans/PhoMT/tokenization/train/train.vi', 'r') as f:
  for line in f:
      train_vi_sentt.append(line.strip())

# Đọc dữ liệu tiếng Anh
with open('/content/drive/MyDrive/NLP/DataTrans/PhoMT/tokenization/train/train.en', 'r') as f:
  for line in f:
      train_en_sentt.append(line.strip())

# Chuyển list văn bản thành mảng numpy
train_vi_sen = np.array(train_vi_sentt[:200000])
train_en_sen = np.array(train_en_sentt[:200000])

In [9]:
test_vi_sen = np.array(train_vi_sentt[250000:275000])
test_en_sen = np.array(train_en_sentt[250000:275000])

In [10]:
train_vi_sen[0:5]

array(['Câu chuyện bắt đầu với buổi lễ đếm ngược .',
       'Ngày 14 , tháng 8 , năm 1947 , gần nửa đêm , ở Bombay , có một phụ nữ sắp lâm bồn .',
       'Cùng lúc , trên khắp đất Ấn , người ta nín thở chờ đợi tuyên ngôn độc lập sau gần hai thập kỷ là thuộc địa của Anh .',
       'Khi đồng hồ điểm thời khắc nửa đêm , một đứa trẻ và hai quốc gia mới , cùng một lúc , ra đời .',
       'Những sự kiện này là nền móng tạo nên " Những đứa con của nửa đêm , " một tiểu thuyết nổi tiếng của nhà văn Anh gốc Ấn Salman Rushdie .'],
      dtype='<U2370')

In [11]:
test_vi_sen[0:5]

array(['Người nghèo là những người bị xâm phạm nhân quyền – mấu chốt của vấn đề là mất phẩm giá , thiếu phẩm giá .',
       'Mọi việc chỉ đơn giản bắt đầu bằng cách trả lại phẩm giá cho họ .',
       'Lúc này , tôi đang ngồi trong một khu ổ chuột ở ngoại ô Gurgaon , ngay gần Delhi , một trong những thành phố bùng lên mạnh mẽ nhất ở Ấn Độ , và tôi đang nói chuyện với công nhân , những người làm việc trong các xí nghiệp dệt may bóc lột ở dưới phố kia , và tôi hỏi họ rằng họ có thông điệp gì muốn gửi gắm .',
       'Họ không nói đến tiền ;',
       "họ chỉ nói , ' Những người chủ lao động không coi chúng tôi là con người , như thể chúng tôi không tồn tại ."],
      dtype='<U709')

In [12]:
train_en_sen[0:5]

array(['It begins with a countdown .',
       'On August 14th , 1947 , a woman in Bombay goes into labor as the clock ticks towards midnight .',
       'Across India , people hold their breath for the declaration of independence after nearly two centuries of British occupation and rule .',
       'And at the stroke of midnight , a squirming infant and two new nations are born in perfect synchronicity .',
       'These events form the foundation of " Midnight \'s Children , " a dazzling novel by the British - Indian author Salman Rushdie .'],
      dtype='<U2567')

## Tách từ cho tiếng việt

### Dùng Vncorenlp để tách từ cho tiếng việt  

In [13]:
# Install the vncorenlp python wrapper
!pip install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 20.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645952 sha256=b9a61ce46b006c701e0023a636ea2844c3272c94bc2af3c2ead5194fa39f283b
  Stored in directory: /root/.cache/pip/wheels/5d/d9/b3/41f6c6b1ab758561fd4aab55dc0480b9d7a131c6aaa573a3fa
Successfully built vncorenlp
--2023-05-04 14:28:57--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[

Test thử một vài câu 

In [14]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')
text = "Đại học Bách Khoa Hà Nội."

word_segmented_text = rdrsegmenter.tokenize(text) 
print(word_segmented_text)

[['Đại_học', 'Bách_Khoa', 'Hà_Nội', '.']]


In [15]:
!wget https://public.vinai.io/PhoBERT_base_transformers.tar.gz
!tar -xzvf PhoBERT_base_transformers.tar.gz

--2023-05-04 14:29:04--  https://public.vinai.io/PhoBERT_base_transformers.tar.gz
Resolving public.vinai.io (public.vinai.io)... 18.64.174.5, 18.64.174.113, 18.64.174.104, ...
Connecting to public.vinai.io (public.vinai.io)|18.64.174.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322405979 (307M) [application/x-tar]
Saving to: ‘PhoBERT_base_transformers.tar.gz’

PhoBERT_base_transf 100%[===================>] 307.47M   134MB/s    in 2.3s    

2023-05-04 14:29:08 (134 MB/s) - ‘PhoBERT_base_transformers.tar.gz’ saved [322405979/322405979]

PhoBERT_base_transformers/
PhoBERT_base_transformers/config.json
PhoBERT_base_transformers/bpe.codes
PhoBERT_base_transformers/model.bin
PhoBERT_base_transformers/dict.txt


Tách từ cho tiếng việt trên tập dữ liệu.

In [16]:
train_vi = []
test_vi = []

In [17]:
for sample in train_vi_sen:
  text = rdrsegmenter.tokenize(sample)
  text = ' '.join([' '.join(x) for x in text])

  train_vi.append(text)

In [18]:
for sample in test_vi_sen:
  text = rdrsegmenter.tokenize(sample)
  text = ' '.join([' '.join(x) for x in text])

  test_vi.append(text)

In [19]:
train_vi = np.array(train_vi)
test_vi = np.array(test_vi)

In [20]:
train_vi[0]

'Câu_chuyện bắt_đầu với buổi lễ đếm ngược .'

In [21]:
test_vi[0]

'Người nghèo là những người bị xâm_phạm nhân_quyền – mấu_chốt của vấn_đề là mất phẩm_giá , thiếu phẩm_giá .'

## Chuẩn bị Tokenizer cho en và vi 

In [22]:
import torch
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import AutoModel, AutoTokenizer, BertTokenizer

tokenizer_en = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer_vi = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Chạy thử trên một số câu tiếng anh và tiếng việt 

In [23]:
test_en_token = tokenizer_en.encode("i am a student")
test_en_token

[101, 1045, 2572, 1037, 3076, 102]

In [24]:
test_vi_token = tokenizer_vi.encode(tf.constant("Ông cụ đi nhanh quá !").numpy().decode('utf8'), return_tensors="tf")
test_vi_token

<tf.Tensor: shape=(1, 8), dtype=int32, numpy=array([[   0,  168, 1591,   57,  485,  204,  381,    2]], dtype=int32)>

## Chuẩn bị dữ liệu đầu vào cho model transformer 

### Cài đặt các hàm cần thiết 

In [25]:
MAX_TOKENS=128

def prepare_batch(en, vi):
    # en = tokenizers.en.tokenize(en)      # Output is ragged.
    en = en[:, :MAX_TOKENS]    # Trim to MAX_TOKENS.
    en = en.to_tensor()  # Convert to 0-padded dense Tensor
    # print(type(en))

    vi = vi[:, :(MAX_TOKENS+1)]
    vi_inputs = vi[:, :-1].to_tensor()
    vi_labels = vi[:, 1:].to_tensor()
    # print(type(vi_inputs))
    
    # en = tokenizers.en.tokenize(en)
    # en = en[:, :(MAX_TOKENS+1)]
    # en_inputs = en[:, :-1].to_tensor()  # Drop the [END] tokens
    # en_labels = en[:, 1:].to_tensor()   # Drop the [START] tokens
    return (en, vi_inputs), vi_labels

In [26]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

In [27]:
def make_batches(ds):
  return (
      ds
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(prepare_batch, tf.data.AUTOTUNE)
      .prefetch(buffer_size=tf.data.AUTOTUNE))

### Thực hiện trên dataset 

In [28]:
train_vi = tf.convert_to_tensor(train_vi)
train_en_sen = tf.convert_to_tensor(train_en_sen)

In [29]:
test_vi = tf.convert_to_tensor(test_vi)
test_en_sen = tf.convert_to_tensor(test_en_sen)

In [30]:
# print(source_test[0])

In [31]:
en_encodeds = []
for sent in train_en_sen:
    en_encoded = tokenizer_en.encode(sent.numpy().decode("utf-8"))
    en_encodeds.append(en_encoded)

train_en_sen = tf.ragged.constant(en_encodeds, dtype=tf.int64)

Token indices sequence length is longer than the specified maximum sequence length for this model (530 > 512). Running this sequence through the model will result in indexing errors


In [32]:
vi_encodeds = []
for sent in train_vi:
    vi_encoded = tokenizer_vi.encode(sent.numpy().decode("utf-8"))
    vi_encodeds.append(vi_encoded)

train_vi = tf.ragged.constant(vi_encodeds, dtype=tf.int64)

In [33]:
vi_encodeds = []
for sent in test_vi:
    vi_encoded = tokenizer_vi.encode(sent.numpy().decode("utf-8"))
    vi_encodeds.append(vi_encoded)

test_vi = tf.ragged.constant(vi_encodeds, dtype=tf.int64)

In [34]:
en_encodeds = []
for sent in test_en_sen:
    en_encoded = tokenizer_en.encode(sent.numpy().decode("utf-8"))
    en_encodeds.append(en_encoded)

test_en_sen = tf.ragged.constant(en_encodeds, dtype=tf.int64)

In [38]:
# dt = tf.data.Dataset.from_tensor_slices((source_train, target_train))
# dt_test = tf.data.Dataset.from_tensor_slices((source_test, target_test))

In [39]:
# print(dt)
# print(dt_test)

In [37]:
train_batches =  make_batches(tf.data.Dataset.from_tensor_slices((train_en_sen, train_vi)))
test_batches =  make_batches(tf.data.Dataset.from_tensor_slices((test_en_sen, test_vi)))

In [40]:
print(train_batches)
print(test_batches)

<_PrefetchDataset element_spec=((TensorSpec(shape=(None, None), dtype=tf.int64, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None)), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>
<_PrefetchDataset element_spec=((TensorSpec(shape=(None, None), dtype=tf.int64, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None)), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>


In [41]:
for (en, vi_), labels in train_batches.take(1):
  break
print(en[0])
print(vi_[0])
print(labels[0])

tf.Tensor(
[ 101 2061 2017 2064 2156 2039 2182 1012  102    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0], shape=(101,), dtype=int64)
tf.Tensor(
[   0 1275    4   88   62  108   25   97    5    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0], shape=(80,), dtype=int64)
tf

### Test 

In [ ]:
# MAX_TOKENS=128

# def prepare_batch(en, vi):
#     # print(en[0])
#     # print(type(en))
#     en = tokenizers.en.tokenize(en)      # Output is ragged.
#     en = en[:, :MAX_TOKENS]    # Trim to MAX_TOKENS.
#     en = en.to_tensor()  # Convert to 0-padded dense Tensor
#     # print(type(en))

#     # print(type(vie))
#     vi_encoded = []
#     for sent in vi:
#       # print(sent)
#       vi_encoded.append(vocab.encode_line(sent.numpy().decode("utf-8"), append_eos=True, add_if_not_exist=False).long().tolist())
#     # vi = pad_sequences(vi_encoded, maxlen=(MAX_TOKENS + 1), dtype="long", value=0, truncating="post", padding="post")
#     vie = tf.ragged.constant(vi_encoded, dtype=tf.int64)
#     vie = vie[:, :(MAX_TOKENS+1)]
#     vi_inputs = vie[:, :-1].to_tensor()
#     vi_labels = vie[:, 1:].to_tensor()
#     # print(type(vi_inputs))
    
#     # en = tokenizers.en.tokenize(en)
#     # en = en[:, :(MAX_TOKENS+1)]
#     # en_inputs = en[:, :-1].to_tensor()  # Drop the [END] tokens
#     # en_labels = en[:, 1:].to_tensor()   # Drop the [START] tokens
#     return en, vi_inputs, vi_labels


# def tf_foo(x, y):
#   en, vi_inputs, vi_labels = tf.py_function(prepare_batch, [x, y], [tf.int64, tf.int64, tf.int64])
#   # print(f"en: {type(en)}, vi_inputs: {type(vi_inputs)}, vi_labels: {type(vi_labels)}")
#   return (en, vi_inputs), vi_labels

In [ ]:
# # def make_batches(ds):
# #   return (
# #       ds
# #       .shuffle(BUFFER_SIZE)
# #       .batch(BATCH_SIZE)
# #       .map(lambda x, y: tf.py_function(prepare_batch, [x, y], 
# #           [(tf.int64, tf.int64), tf.int64]), 
# #           tf.data.AUTOTUNE)
# #       .prefetch(buffer_size=tf.data.AUTOTUNE))

# def make_batches(ds):
#   return (
#       ds
#       .shuffle(BUFFER_SIZE)
#       .batch(BATCH_SIZE)
#       .map(tf_foo, tf.data.AUTOTUNE)
#       .prefetch(buffer_size=tf.data.AUTOTUNE))

## Define the components

In [42]:
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)
  
  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1) 

  return tf.cast(pos_encoding, dtype=tf.float32)

In [43]:
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True) 
    self.pos_encoding = positional_encoding(length=2048, depth=d_model)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x


In [ ]:
# print(tokenizer_en.vocab_size)
# print(tokenizer_vi.vocab_size)

In [ ]:
# embed_vi = PositionalEmbedding(vocab_size=tokenizer_vi.vocab_size, d_model=512)
# embed_en = PositionalEmbedding(vocab_size=tokenizer_en.vocab_size, d_model=512)

# vi_emb = embed_vi(vi_)
# en_emb = embed_en(en)

In [45]:
class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

In [46]:
class CrossAttention(BaseAttention):
  def call(self, x, context):
    attn_output, attn_scores = self.mha(
        query=x,
        key=context,
        value=context,
        return_attention_scores=True)
   
    # Cache the attention scores for plotting later.
    self.last_attn_scores = attn_scores

    x = self.add([x, attn_output])
    x = self.layernorm(x)

    return x

In [ ]:
# sample_ca = CrossAttention(num_heads=2, key_dim=512)

# print(vi_emb.shape)
# print(en_emb.shape)
# print(sample_ca(en_emb, vi_emb).shape)

In [48]:
class GlobalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [ ]:
# sample_gsa = GlobalSelfAttention(num_heads=2, key_dim=512)

# print(en_emb.shape)
# print(sample_gsa(en_emb).shape)

In [49]:
class CausalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x,
        use_causal_mask = True)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [ ]:
# sample_csa = CausalSelfAttention(num_heads=2, key_dim=512)

# print(vi_emb.shape)
# print(sample_csa(vi_emb).shape)

In [ ]:
# out1 = sample_csa(embed_en(en[:, :3])) 
# out2 = sample_csa(embed_en(en))[:, :3]

# tf.reduce_max(abs(out1 - out2)).numpy()

In [50]:
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x) 
    return x

In [ ]:
# sample_ffn = FeedForward(512, 2048)

# print(en_emb.shape)
# print(sample_ffn(en_emb).shape)

In [51]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
    super().__init__()

    self.self_attention = GlobalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x):
    x = self.self_attention(x)
    x = self.ffn(x)
    return x

In [ ]:
# sample_encoder_layer = EncoderLayer(d_model=512, num_heads=8, dff=2048)

# print(en_emb.shape)
# print(sample_encoder_layer(en_emb).shape)

In [52]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads,
               dff, vocab_size, dropout_rate=0.1):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(
        vocab_size=vocab_size, d_model=d_model)

    self.enc_layers = [
        EncoderLayer(d_model=d_model,
                     num_heads=num_heads,
                     dff=dff,
                     dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x):
    # `x` is token-IDs shape: (batch, seq_len)
    x = self.pos_embedding(x)  # Shape `(batch_size, seq_len, d_model)`.
    
    # Add dropout.
    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x)

    return x  # Shape `(batch_size, seq_len, d_model)`.

In [ ]:
# # Instantiate the encoder.
# sample_encoder = Encoder(num_layers=4,
#                          d_model=512,
#                          num_heads=8,
#                          dff=2048,
#                          vocab_size=40000)

# sample_encoder_output = sample_encoder(en, training=False)

# # Print the shape.
# print(en.shape)
# print(sample_encoder_output.shape)  # Shape `(batch_size, input_seq_len, d_model)`.

In [53]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self,
               *,
               d_model,
               num_heads,
               dff,
               dropout_rate=0.1):
    super(DecoderLayer, self).__init__()

    self.causal_self_attention = CausalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)
    
    self.cross_attention = CrossAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x, context):
    x = self.causal_self_attention(x=x)
    x = self.cross_attention(x=x, context=context)

    # Cache the last attention scores for plotting later
    self.last_attn_scores = self.cross_attention.last_attn_scores

    x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
    return x

In [ ]:
# sample_decoder_layer = DecoderLayer(d_model=512, num_heads=8, dff=2048)

# sample_decoder_layer_output = sample_decoder_layer(
#     x=vi_emb, context=en_emb)

# print(en_emb.shape)
# print(vi_emb.shape)
# print(sample_decoder_layer_output.shape) # `(batch_size, seq_len, d_model)`

In [54]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size,
               dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size,
                                             d_model=d_model)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dec_layers = [
        DecoderLayer(d_model=d_model, num_heads=num_heads,
                     dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]

    self.last_attn_scores = None

  def call(self, x, context):
    # `x` is token-IDs shape (batch, target_seq_len)
    x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

    x = self.dropout(x)

    for i in range(self.num_layers):
      x  = self.dec_layers[i](x, context)

    self.last_attn_scores = self.dec_layers[-1].last_attn_scores

    # The shape of x is (batch_size, target_seq_len, d_model).
    return x

In [ ]:
# # Instantiate the decoder.
# sample_decoder = Decoder(num_layers=4,
#                          d_model=512,
#                          num_heads=8,
#                          dff=2048,
#                          vocab_size=70000)

# output = sample_decoder(
#     x=vi_,
#     context=en_emb)

# # Print the shapes.
# print(vi_.shape)
# print(en_emb.shape)
# print(output.shape)

In [ ]:
# sample_decoder.last_attn_scores.shape  # (batch, heads, target_seq, input_seq)

In [55]:
class Transformer(tf.keras.Model):
  def __init__(self, *, num_layers, d_model, num_heads, dff,
               input_vocab_size, target_vocab_size, dropout_rate=0.1):
    super().__init__()
    self.encoder = Encoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=input_vocab_size,
                           dropout_rate=dropout_rate)

    self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=target_vocab_size,
                           dropout_rate=dropout_rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inputs):
    # To use a Keras model with `.fit` you must pass all your inputs in the
    # first argument.
    context, x  = inputs

    context = self.encoder(context)  # (batch_size, context_len, d_model)

    x = self.decoder(x, context)  # (batch_size, target_len, d_model)

    # Final linear layer output.
    logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)

    try:
      # Drop the keras mask, so it doesn't scale the losses/metrics.
      # b/250038731
      del logits._keras_mask
    except AttributeError:
      pass

    # Return the final output and the attention weights.
    return logits

In [56]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

In [57]:
transformer_ = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=tokenizer_en.vocab_size,
    target_vocab_size=tokenizer_vi.vocab_size,
    dropout_rate=dropout_rate)

In [58]:
output = transformer_((en, vi_))

print(en.shape)
print(vi_.shape)
print(output.shape)

(64, 101)
(64, 80)
(64, 80, 64000)


In [59]:
attn_scores = transformer_.decoder.dec_layers[-1].last_attn_scores
print(attn_scores.shape)  # (batch, heads, target_seq, input_seq)

(64, 8, 80, 101)


In [60]:
transformer_.summary()

Model: "transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Encoder)           multiple                  6545664   
                                                                 
 decoder (Decoder)           multiple                  12941824  
                                                                 
 dense_16 (Dense)            multiple                  8256000   
                                                                 
Total params: 27,743,488
Trainable params: 27,743,488
Non-trainable params: 0
_________________________________________________________________


In [61]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [62]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [ ]:
plt.plot(learning_rate(tf.range(40000, dtype=tf.float32)))
plt.ylabel('Learning Rate')
plt.xlabel('Train Step')

In [63]:
def masked_loss(label, pred):
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss


def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 0

  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [64]:
transformer_.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy])

In [65]:
import os

In [66]:
checkpoint_path = "/content/drive/MyDrive/NLP/DataTrans/model/TransEnVi.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
transformer_.fit(train_batches,
                epochs=15,
                validation_data=test_batches,
                callbacks=[cp_callback])

Epoch 1/15
  67/3125 [..............................] - ETA: 40:44 - loss: 11.0071 - masked_accuracy: 0.0240

## Load model và test thử. 

In [ ]:
class Translator(tf.Module):
  def __init__(self, tokenizer_en, tokenizer_vi, transformer):
    self.tokenizer_en = tokenizer_en
    self.tokenizer_vi = tokenizer_vi
    self.transformer = transformer

  def __call__(self, sentence, max_length=MAX_TOKENS):
    # The input sentence is English, hence adding the `[START]` and `[END]` tokens.
    # sentence = np.array([sentence])
    sentence = tf.convert_to_tensor(sentence)
    
    assert isinstance(sentence, tf.Tensor)

    # if len(sentence.shape) == 0:
    #   sentence = sentence[tf.newaxis]
    # print(sentence)
    # sentence = tf.convert_to_tensor(sentence)
    
    sentence = self.tokenizer_en.encode(sentence.numpy().decode("utf-8"))
    sentence = np.array(sentence)
    sentence = tf.convert_to_tensor(sentence)
    # print(sentence) 
    encoder_input = sentence
    encoder_input = tf.expand_dims(encoder_input, 0)

    # As the output language is VietNamese, initialize the output with the
    # VietNamese `[START]` token.
    # start_end = self.tokenizer_vi.tokenize([''])[0]
    # start_end = self.tokenizer_vi.build_inputs_with_special_tokens([])
    # start = start_end[0][tf.newaxis]
    # end = start_end[1][tf.newaxis]
    start = tf.constant([self.tokenizer_vi.cls_token_id], dtype=tf.int64)
    end = tf.constant([self.tokenizer_vi.sep_token_id], dtype=tf.int64)

    # `tf.TensorArray` is required here (instead of a Python list), so that the
    # dynamic-loop can be traced by `tf.function`.
    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
    output_array = output_array.write(0, start)


    for i in tf.range(max_length):
      output = tf.transpose(output_array.stack())
      predictions = self.transformer([encoder_input, output], training=False)


      # Select the last token from the `seq_len` dimension.
      predictions = predictions[:, -1:, :]  # Shape `(batch_size, 1, vocab_size)`.

      predicted_id = tf.argmax(predictions, axis=-1)

      # Concatenate the `predicted_id` to the output which is given to the
      # decoder as its input.
      output_array = output_array.write(i+1, predicted_id[0])

      if predicted_id == end:
        break

    output = tf.transpose(output_array.stack())
    # The output shape is `(1, tokens)`.
    # text = tokenizer_vi.detokenize(output)[0]  # Shape: `()`.
    # tokens = tokenizer_vi.lookup(output)[0])

    text = self.tokenizer_vi.decode(output.numpy()[0], skip_special_tokens=True)  # Shape: `()`.

    tokens = self.tokenizer_vi.convert_ids_to_tokens(output.numpy()[0])

    # `tf.function` prevents us from using the attention_weights that were
    # calculated on the last iteration of the loop.
    # So, recalculate them outside the loop.
    self.transformer([encoder_input, output[:,:-1]], training=False)
    attention_weights = self.transformer.decoder.last_attn_scores

    return text, tokens, attention_weights

In [ ]:
def print_translation(sentence, tokens, ground_truth):
  print(f'{"Input:":15s}: {sentence}')
  print(f'{"Prediction":15s}: {tokens}')
  print(f'{"Ground truth":15s}: {ground_truth}')

In [ ]:
model = Transformer(num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=tokenizer_en.vocab_size,
    target_vocab_size=tokenizer_vi.vocab_size,
    dropout_rate=dropout_rate)

model.load_weights(checkpoint_path)


In [ ]:
translator = Translator(tokenizer_en, tokenizer_vi, model)

In [ ]:
sentence = 'How are you ?'
ground_truth = 'Mày thấy có khỏe chứ ?'

# print(tf.convert_to_tensor(sentence))

translated_text, translated_tokens, attention_weights = translator(
    sentence)
print_translation(sentence, translated_text, ground_truth)

# <class 'tensorflow.python.framework.ops.EagerTensor'>

In [ ]:
print_translation(sentence, translated_text, ground_truth)